In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../src')
import geopandas as gpd
from ipyleaflet import Map, GeoData, LayersControl, WidgetControl
from ipywidgets import IntSlider, VBox, HTML, Button, HBox
import ipyleaflet as ipyl
import shapely
import ee

from gee import initialize_ee_with_credentials, get_s2_hsv_median, get_ee_image_url, get_planet_rgb_median, get_planet_ndvi_median, get_planet_hsv_median
from ui import BASEMAP_TILES

initialize_ee_with_credentials()

ra_polygons = gpd.GeoDataFrame(geometry=[gpd.read_file(
    "gs://demeter-labs/tea/geometries/ra_data/subsets/central_america_coffee_polygons.geojson").buffer(0.0001).union_all()])
v2_det_5 = gpd.GeoDataFrame(
    geometry=gpd.read_parquet("gs://demeter-labs/coffee/detections/tile_classifier_predictions_v0_mesoamerica_posw1.0_prob_0.9_postprocess.parquet").geometry)
v2_det_5 = gpd.GeoDataFrame(
    geometry=[v2_det_5.buffer(0.0001).union_all()], crs=v2_det_5.crs).explode(index_parts=True)
v2_det_5['utm_zone'] = v2_det_5.geometry.centroid.apply(lambda x: int(((x.x + 180) / 6) + 1))
v2_det_5['epsg'] = v2_det_5['utm_zone'].apply(lambda x: 32600 + x if v2_det_5.geometry.centroid.y.mean() >= 0 else 32700 + x)
v2_det_5['area'] = v2_det_5.apply(
    lambda row: gpd.GeoDataFrame(geometry=[row.geometry], crs=v2_det_5.crs).to_crs(epsg=row.epsg).geometry.iloc[0].area / 10000, axis=1)


/tmp/ipykernel_2478671/4004352101.py:19: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  "gs://demeter-labs/tea/geometries/ra_data/subsets/central_america_coffee_polygons.geojson").buffer(0.0001).union_all()])
/tmp/ipykernel_2478671/4004352101.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometry=[v2_det_5.buffer(0.0001).union_all()], crs=v2_det_5.crs).explode(index_parts=True)
/tmp/ipykernel_2478671/4004352101.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  v2_det_5['utm_zone'] = v2_det_5.geometry.centroid.apply(lambda x: int(((x.x + 180) / 6) + 1))
/tmp/ipykerne

KeyboardInterrupt: 

In [3]:
from shapely.prepared import prep
import ipywidgets as ipyw
false_negatives = gpd.overlay(ra_polygons, v2_det_5, how='difference')
ra_polygons = ra_polygons.set_crs(epsg=4326)
ra_polygons_sindex = ra_polygons.sindex


BOUNDARY_PATH = "/home/christopher.x.ren/earth-index-ml/places/ra_mesoamerica.geojson"
BOUNDARY = gpd.read_file(BOUNDARY_PATH)
boundary_geom = ee.Geometry(shapely.geometry.mapping(BOUNDARY.geometry.iloc[0]))

hsv_median = get_s2_hsv_median(boundary_geom, '2023-01-01', '2024-12-31')
planet_rgb = get_planet_rgb_median(boundary_geom, '2023-01-01', '2024-12-31')
planet_ndvi = get_planet_ndvi_median(boundary_geom, '2023-01-01', '2024-12-31')
planet_hsv = get_planet_hsv_median(boundary_geom, '2023-01-01', '2024-12-31')

# Get tile URLs
hsv_url = get_ee_image_url(hsv_median, {
    'min': [0, 0, 0],
    'max': [1, 1, 1],
    'bands': ['hue', 'saturation', 'value']
})
rgb_url = get_ee_image_url(planet_rgb, {
    'min': [0, 0, 0],
    'max': [2000, 2000, 2000],
    'bands': ['R', 'G', 'B']
})
ndvi_url = get_ee_image_url(planet_ndvi, {
    'min': -1,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
})
planet_hsv_url = get_ee_image_url(planet_hsv, {
    'min': [0, 0, 0],
    'max': [1, 1, 1],
    'bands': ['hue', 'saturation', 'value']
})

BASEMAP_TILES['HSV_MEDIAN'] = hsv_url
BASEMAP_TILES['PLANET_RGB'] = rgb_url
BASEMAP_TILES['PLANET_NDVI'] = ndvi_url
BASEMAP_TILES['PLANET_HSV'] = planet_hsv_url

current_basemap = 'GOOGLE_HYBRID'
basemap_layer = ipyl.TileLayer(url=BASEMAP_TILES[current_basemap], no_wrap=True, name='basemap')
m = Map(basemap=basemap_layer, center=(-2, 120), zoom=5, layout={'height':'600px'}, scroll_wheel_zoom=True)

ra_polygons_layer = GeoData(
    geo_dataframe=ra_polygons,
    style={'color': '#4daf4a', 'fillOpacity': 0},
    name='RA Polygons'
)
m.add_layer(ra_polygons_layer)

v2_det_layer = GeoData(
    geo_dataframe=v2_det_5,
    style={'color': '#377eb8', 'fillOpacity': 0},
    name='V2 Detections'
)
m.add_layer(v2_det_layer)

false_negatives_layer = GeoData(
    geo_dataframe=false_negatives,
    style={'color': '#e41a1c', 'fillOpacity': 0},
    name='False Negatives'
)
m.add_layer(false_negatives_layer)

# Add layer control
layer_control = LayersControl(position='topright')
m.add_control(layer_control)

# Add legend
legend_html = """
<div style="background-color: white; padding: 10px; border-radius: 5px;">
    <h4 style="margin-bottom: 5px; font-size: 18px;">Legend</h4>
    <div style="display: flex; align-items: center; margin: 5px 0;">
        <div style="width: 20px; height: 20px; background-color: #4daf4a; margin-right: 5px; opacity: 1.0; border: 1px solid #4daf4a;"></div>
        <span style="font-size: 16px;">RA Polygons</span>
    </div>
    <div style="display: flex; align-items: center; margin: 5px 0;">
        <div style="width: 20px; height: 20px; background-color: #377eb8; margin-right: 5px; opacity: 1.0;"></div>
        <span style="font-size: 16px;">V2 Detections</span>
    </div>
    <div style="display: flex; align-items: center; margin: 5px 0;">
        <div style="width: 20px; height: 20px; background-color: #984ea3; margin-right: 5px; opacity: 1.0;"></div>
        <span style="font-size: 16px;">Planet Detections</span>
    </div>
    <div style="display: flex; align-items: center; margin: 5px 0;">
        <div style="width: 20px; height: 20px; background-color: #e41a1c; margin-right: 5px; opacity: 1.0;"></div>
        <span style="font-size: 16px;">False Negatives</span>
    </div>
</div>
"""
legend = HTML(value=legend_html)
legend_control = WidgetControl(widget=legend, position='bottomright')
m.add_control(legend_control)

# Add area filter slider
min_area = v2_det_5['area'].min()
area_slider = IntSlider(
    value=int(min_area),
    min=int(min_area),
    max=200,
    step=1,
    description='Min Area (ha):',
    continuous_update=False
)

# def update_layers(change):
#     import cProfile
#     import pstats
#     import io
    
#     pr = cProfile.Profile()
#     pr.enable()
    
#     # Filter detections based on area
#     filtered_v2_det = v2_det_5[v2_det_5['area'] >= change['new']]
    
#     # Recalculate false negatives
#     new_false_negatives = gpd.overlay(
#         ra_polygons, filtered_v2_det, how='difference')
    
#     # Update layers
#     v2_det_layer.geo_dataframe = filtered_v2_det
    
#     # Remove old false negatives layer and add new one
#     for layer in m.layers:
#         if layer.name == 'False Negatives':
#             m.remove_layer(layer)
#             break

#     new_false_negatives_layer = GeoData(
#         geo_dataframe=new_false_negatives,
#         style={'color': '#e41a1c', 'fillOpacity': 0},
#         name='False Negatives'
#     )
#     m.add_layer(new_false_negatives_layer)
    
#     pr.disable()
#     s = io.StringIO()
#     ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
#     ps.print_stats()
#     print(s.getvalue())

def update_layers(change):
    """Update layers based on area threshold change."""
    # Filter detections based on area
    filtered_v2_det = v2_det_5[v2_det_5['area'] >= change['new']]
    
    if len(filtered_v2_det) == 0:
        # If no detections pass the filter, everything is a false negative
        new_false_negatives = ra_polygons.copy()
    else:
        # Get the total bounds of filtered detections
        bounds = filtered_v2_det.total_bounds
        
        # Use spatial index to find reference polygons that might intersect
        candidate_indices = list(ra_polygons_sindex.intersection(bounds))
        candidates = ra_polygons.iloc[candidate_indices]
        
        # Prepare filtered geometry for faster operations
        prepared_filtered = prep(filtered_v2_det.union_all())
        
        # Further refine candidates using prepared geometry
        mask = ~candidates.geometry.apply(prepared_filtered.contains)
        candidates = candidates[mask]
        
        if len(candidates) == 0:
            new_false_negatives = gpd.GeoDataFrame(
                geometry=[], crs=ra_polygons.crs
            )
        else:
            # Only run expensive difference operation on real candidates
            new_false_negatives = gpd.overlay(
                candidates, filtered_v2_det, how='difference'
            )
    
    for idx, layer in enumerate(m.layers):
        if layer.name == 'False Negatives':
            m.remove_layer(layer)
    
    # Add new layer if there are false negatives
    if len(new_false_negatives) > 0:
        new_false_negatives_layer = GeoData(
            geo_dataframe=new_false_negatives,
            style={'color': '#e41a1c', 'fillOpacity': 0.5},
            name='False Negatives'
        )
        m.add_layer(new_false_negatives_layer)

    # Update filtered detections layer
    v2_det_layer.geo_dataframe = filtered_v2_det


area_slider.observe(update_layers, names='value')

# Add buttons
toggle_basemap_button = Button(description=f'Basemap: {current_basemap}')
google_maps_button = Button(description='Google Maps')

# Add label for coordinates
label = ipyw.Label()
display(label)

def handle_mouse_move(**kwargs):
    lat, lon = kwargs.get('coordinates')
    label.value = f'Lat/lon: {lat:.4f}, {lon:.4f}'

m.on_interaction(handle_mouse_move)

def toggle_basemap(b):
    global current_basemap
    basemap_keys = list(BASEMAP_TILES.keys())
    current_idx = basemap_keys.index(current_basemap)
    next_idx = (current_idx + 1) % len(basemap_keys)
    current_basemap = basemap_keys[next_idx]
    
    # Update basemap layer
    basemap_layer.url = BASEMAP_TILES[current_basemap]
    toggle_basemap_button.description = f'Basemap: {current_basemap}'

def google_maps_click(b):
    lat, lon = label.value.split(': ')[1].split(',')  # Extract coordinates from label
    lat = float(lat)
    lon = float(lon.split('.')[0] + '.' + lon.split('.')[1])
    import webbrowser
    url = f"https://www.google.com/maps/search/?api=1&query={lat},{lon}"
    webbrowser.open(url, new=2, autoraise=True)

toggle_basemap_button.on_click(toggle_basemap)
google_maps_button.on_click(google_maps_click)

# Display map and controls in a vertical box layout
display(VBox([
    m,
    area_slider,
    HBox([
        google_maps_button,
        toggle_basemap_button
    ])
]))


Label(value='')